In [ ]:
from rasterstats import zonal_stats  # https://pythonhosted.org/rasterstats/manual.html#zonal-statistics
import os
import glob
import pandas as pd
import geopandas as gpd
import rasterio as rio

In [ ]:
# Source Raster paths and shoreline buffer polygons
raster_toa_path = '/Users/arbailey/Google Drive/My Drive/sargassum/s2toa_classified'
raster_sr_path = '/Users/arbailey/Google Drive/My Drive/sargassum/s2sr_classified'
shore_buffer_path = '/Users/arbailey/natcap/idb/data/work/sargassum/shore_segments'
shore_buffer_file = 'shoreQR_segments_20210331_buffer100m.shp'
shore_buffer = os.path.join(shore_buffer_path, shore_buffer_file)

In [ ]:
#. Use Zonal stats to get count of each Land cover type (Sargassum present, absent, & no data)
cmap = {1: 'pr', -1: 'nd', 0: 'ab', -9999: 'ab2'}  # convert numeric values to present/absent/no data 2-letter codes
def sarg_segments(shore_polys, sarg_raster, affine=None):
    zs = zonal_stats(shore_polys, sarg_raster,
            stats="count",
            all_touched=True,
            categorical=True, category_map=cmap,
            geojson_out=True,
            affine=affine)
    return zs

# Convert Zonal Stats GeoJSON result to Geodataframe for some field manipulations and joining
def sargzs_gdf(zs_geojson, imgdate=None, imgtype=None):
    zs_gdf = gpd.GeoDataFrame.from_features(zs_geojson)
    zs_gdf.drop(columns=['geometry','fid', 'type_geomo','length_km', 'shore_desc', 'desc_abbre'], inplace=True, errors='ignore')
    # Assign no data pixels to absent (because they are pre-masked as non-sargassum areas )
    if 'ab2' in zs_gdf.columns:
        zs_gdf['ab'] = zs_gdf['ab'] + zs_gdf['ab2']
        zs_gdf.drop(columns=['ab2'], inplace=True, errors='ignore')
    if imgdate:
        zs_gdf['imgdate'] = imgdate
    if imgtype:
        zs_gdf.rename(columns={"pr": imgtype + "_pr", 
                           "nd": imgtype + "_nd", 
                           "ab": imgtype + "_ab",
                           "count": imgtype + "_cnt"}, inplace=True)
    return zs_gdf

In [ ]:
# Example using rasterio array, get different numbers from pulling from VRT directly?!
# with rio.open(os.path.join(raster_toa_path, '20190626T160839_mosaic.vrt')) as src:
#     affine = src.transform
#     array = src.read(1)
# # zs = zonal_stats(shore_buffer, array, affine=affine)
# zs = sarg_segments(shore_buffer, array, affine)
# print(affine)
# zs[0]

In [ ]:
# TOA -- Calculate Zonal Stats & create stacked DF

# Get list of all mosaic rasters
os.chdir(raster_toa_path)
# toa_mosaics = [f for f in glob.glob('*_mosaic.vrt')] 
toa_mosaics = [f for f in glob.glob('*_mosaic_nd0.vrt')]
toa_mosaics.sort()
print(toa_mosaics)

dfs = []
print('Calculating TOA Sargassum and Nearshore Zonal Stats......')
for mosaic in toa_mosaics:
    image_date = mosaic[0:8]
    toa_zs = sarg_segments(shore_buffer, mosaic)
    toa_shore_df = sargzs_gdf(toa_zs,image_date,'toa')
    dfs.append(toa_shore_df)
toa_shore_df = pd.concat(dfs)
toa_shore_df

In [ ]:
# SR -- Calculate Zonal Stats & create stacked DF

# Get list of all mosaic rasters
os.chdir(raster_sr_path)
# sr_mosaics = [f for f in glob.glob('*_mosaic.vrt')]
sr_mosaics = [f for f in glob.glob('*_mosaic_nd0.vrt')]
sr_mosaics.sort()
print(sr_mosaics)

dfs = []
print('Calculating SR Sargassum and Nearshore Zonal Stats......')
for mosaic in sr_mosaics:
    image_date = mosaic[0:8]
    sr_zs = sarg_segments(shore_buffer, mosaic)
    sr_shore_df = sargzs_gdf(sr_zs,image_date,'sr')
    dfs.append(sr_shore_df)
sr_shore_df = pd.concat(dfs)
sr_shore_df


In [ ]:
# DB-style inner join SR and TOA 
joined_shore_df = pd.merge(right=sr_shore_df, left=toa_shore_df, how='inner',on=['seg_id','imgdate'])  # uses fields in common by default
joined_shore_df = joined_shore_df.fillna(0, )
joined_shore_df

In [ ]:
joined_shore_df['pr_diff'] = joined_shore_df['toa_pr'] - joined_shore_df['sr_pr']
joined_shore_df

In [ ]:
joined_shore_df['pr_diff'].plot.hist(bins=100, figsize=(20,12))

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")
sns.set_theme(style="darkgrid")

f, ax = plt.subplots(figsize=(20, 12))
sns.despine(f)

sns.histplot(
    joined_shore_df,
    x="pr_diff",
)

In [ ]:
f, ax = plt.subplots(figsize=(20, 12))
sns.set_theme(style="darkgrid")
ax.set_xticks([-300, -200, -100,0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200])
ax.set_yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
ax.set_ylabel('Cumulative Percent')
sns.histplot(
    joined_shore_df,
    x="pr_diff",
    cumulative = True,
    stat = 'probability',
)

In [ ]:
#### Output
output_path = '/Users/arbailey/natcap/idb/data/work/sargassum/ip'
output_csv = os.path.join(output_path, 'shore100m_sargassum_stats.csv')
print('output dataframe to ' + output_csv)
joined_shore_df.to_csv(path_or_buf=output_csv, index=False)